<a href="https://colab.research.google.com/github/cereal-d3v/Veg-Vibe/blob/main/frontend_Veg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies as needed:
! pip install kagglehub[pandas-datasets] --quiet
import kagglehub
from kagglehub import KaggleDatasetAdapter


# Download latest version
path = kagglehub.dataset_download("akeshkumarhp/vegan-recipies-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/vegan-recipies-dataset


In [3]:
from google.colab import userdata

try:
    kaggle_secret = userdata.get('kaggle.json')
    if kaggle_secret:
        print("KAGGLE_JSON secret found.")
    else:
        print("KAGGLE_JSON secret not found or is empty.")
except Exception as e:
    print(f"An error occurred while accessing KAGGLE_JSON secret: {e}")

An error occurred while accessing KAGGLE_JSON secret: Secret kaggle.json does not exist.


In [9]:
ls '/kaggle/input/vegan-recipies-dataset/'

data.csv


In [10]:
path = '/kaggle/input/vegan-recipies-dataset/data.csv'

In [11]:
import pandas as pd
# Load the latest version
df = pd.read_csv(path)

print("First 5 records:")
df.head()

First 5 records:


,name,collection,recipie_collection_idx,image,descripition,ingredients,steps,Neutretion
0,Chickpea & Potato Curry,collection/vegan-recipes/,1,https://www.tasteofhome.com/wp-content/uploads...,\n\n1\n/\n92\n\n\n\n\n\n\n\nChickpea & Potato ...,"['1 tablespoon canola oil', '1 medium onion, c...","['In a large skillet, heat oil over medium-hig...","<p>1-1/4 cups chickpea mixture: 240 calories, ..."
1,Crispy Tofu with Black Pepper Sauce,collection/vegan-recipes/,2,https://www.tasteofhome.com/wp-content/uploads...,\n\n2\n/\n92\n\n\n\n\n\n\n\n\nCrispy Tofu with...,"['2 tablespoons reduced-sodium soy sauce', '2 ...",['Mix the first 4 ingredients. Mince white par...,"<p>1 cup: 316 calories, 24g fat (2g saturated ..."
2,Over-the-Rainbow Minestrone,collection/vegan-recipes/,3,https://www.tasteofhome.com/wp-content/uploads...,\n\n3\n/\n92\n\n\n\n\n\n\n\nOver-the-Rainbow M...,"['1/2 pound Swiss chard', '2 tablespoons olive...",['Cut stems from chard; chop stems and leaves ...,"<p>1-1/2 cups: 231 calories, 7g fat (1g satura..."
3,Simple Vegetarian Slow-Cooked Beans,collection/vegan-recipes/,4,https://www.tasteofhome.com/wp-content/uploads...,\n\n4\n/\n92\n\n\n\n\n\n\n\nTaste of HomeSimpl...,['4 cans (15-1/2 ounces each) great northern b...,"['In a 3-qt. slow cooker, combine the first 7 ...","<p>3/4 cup: 229 calories, 3g fat (0 saturated ..."
4,Vegetarian Black Bean Pasta,collection/vegan-recipes/,5,https://www.tasteofhome.com/wp-content/uploads...,\n\n5\n/\n92\n\n\n\n\n\n\n\nTaste of HomeVeget...,"['9 ounces uncooked whole wheat fettuccine', '...",['Cook fettuccine according to package directi...,"<p>1-1/4 cups: 255 calories, 3g fat (0 saturat..."


**Content-Based Recommender Skeleton**

In [12]:
!pip install -q gradio nltk scikit-learn pandas

import pandas as pd
import re
import nltk
import gradio as gr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# Example vegan recipe data (replace with uploaded file later)
data = {
    "title": [
        "Vegan Mac and Cheese",
        "Chickpea Salad",
        "Tofu Stir Fry",
        "Lentil Soup",
        "Vegan Pancakes"
    ],
    "ingredients": [
        "pasta, vegan cheese, almond milk, garlic, nutritional yeast",
        "chickpeas, cucumber, tomato, olive oil, lemon juice",
        "tofu, soy sauce, garlic, broccoli, bell pepper",
        "lentils, carrots, celery, onion, garlic, vegetable broth",
        "flour, almond milk, baking powder, maple syrup"
    ]
}
df = pd.DataFrame(data)

def clean_text(text):
    text = re.sub(r'\d+', '', str(text))
    text = re.sub(r'[^\w\s]', '', text).lower()
    return ' '.join(word for word in text.split() if word not in stop_words)

df['cleaned'] = df['ingredients'].apply(clean_text)
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['cleaned'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def get_recommendations(input_ingredients, topn=3):
    cleaned_input = clean_text(input_ingredients)
    input_vec = tfidf.transform([cleaned_input])
    sim_scores = linear_kernel(input_vec, tfidf_matrix).flatten()
    top_indices = sim_scores.argsort()[-topn:][::-1]
    recs = df.iloc[top_indices][['title', 'ingredients']]
    return recs.to_string(index=False)

def chatbot_interface(user_input):
    return f"🍽️ Here are vegan recipes you can make:\n\n{get_recommendations(user_input)}"

gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(label="Enter your available ingredients"),
    outputs=gr.Textbox(label="Recipe Suggestions"),
    title="🥬 Vegan Recipe Recommender",
    description="Input ingredients in your kitchen. Get custom vegan recipes!"
).launch()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8a416eb4564caaccb8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
